In [1]:
import json
import pandas as pd
import requests
import yfinance as yf
from datetime import date, timedelta
import numpy as np

In [2]:
stock = "MSFT"
infer_request = requests.get(f"http://127.0.0.1:5000/inference/{stock}")
infer_string = json.loads(infer_request.text)
new_datapoint = []


In [3]:
td = date.today()
infer_dict = {}
infer_array = []
i = 0
for key, val in infer_string.items():
    infer_dict['Date'] = td
    infer_dict['Predicted'] = val
    infer_array.append(infer_dict)
    infer_dict = {}
    if i != 0:
        new_datapoint.append(td)
    # print(td, val)
    td = td+timedelta(days=1)
    while td.weekday() in [5, 6]:
        td = td+timedelta(days=1)
    i += 1
# infer_array
infer_dataframe = pd.DataFrame(infer_array)
infer_dataframe


,Date,Predicted
0,2023-04-24,266.412861
1,2023-04-25,266.888085
2,2023-04-26,267.182780
3,2023-04-27,267.289946
4,2023-04-28,267.215979


In [4]:
new_datapoint

[datetime.date(2023, 4, 25),
 datetime.date(2023, 4, 26),
 datetime.date(2023, 4, 27),
 datetime.date(2023, 4, 28)]

In [5]:
stock = "MSFT"
request = requests.get(f"http://127.0.0.1:5000/lstm_data_view/{stock}")
js_string = json.loads(request.text)


In [6]:
# Use pandas.DataFrame.from_dict() to Convert JSON to DataFrame
predict_df = pd.read_json(request.text, orient='index')
predict_df.columns = ["Actual", "Date", "Predicted"]
predict_df.Date = predict_df.Date.dt.date
predict_df.tail(5)


,Actual,Date,Predicted
842,288.80,2023-04-17,261.668
843,288.37,2023-04-18,262.604
844,288.45,2023-04-19,263.485
845,286.11,2023-04-20,264.308
846,285.76,2023-04-21,265.076


In [7]:
predict_df = pd.concat([predict_df, infer_dataframe], ignore_index=True)


In [8]:
predict_df.tail(6)


,Actual,Date,Predicted
846,285.76,2023-04-21,265.076000
847,NaN,2023-04-24,266.412861
848,NaN,2023-04-25,266.888085
849,NaN,2023-04-26,267.182780
850,NaN,2023-04-27,267.289946
851,NaN,2023-04-28,267.215979


In [19]:
st = yf.Ticker(stock)
periods = {"1d": "1D",
           "5d": "5D",
           "1mo": "1M",
           "6mo": "6M",
           "ytd": "YTD",
           "1y": "1Y",
           "5y": "5Y",
           "max": "Max"}
intervals = {"1d": "1m",
             "5d": "15m",  # 15m
             "1mo": "90m",
             "6mo": "1d",
             "ytd": "1d",
             "1y": "1d",
             "5y": "1wk",
             "max": "3mo"}
period = "6mo"
interval = intervals[period]
# print(interval)
# Create a line graph
df = st.history(period=(period), interval=interval)
df = df.reset_index()

df.columns = ['Date-Time']+list(df.columns[1:])
df["Date"] = df["Date-Time"].dt.date


In [20]:


df = pd.concat([df, pd.DataFrame(new_datapoint, columns=['Date'])])


In [21]:
df = df.reset_index().drop(columns=["index"])

In [23]:
df_date = pd.to_datetime(df['Date'])
df_date = df_date.dt.tz_localize('US/Eastern').dt.tz_convert('US/Eastern')
df_date


0     2022-10-24 00:00:00-04:00
1     2022-10-25 00:00:00-04:00
2     2022-10-26 00:00:00-04:00
3     2022-10-27 00:00:00-04:00
4     2022-10-28 00:00:00-04:00
                 ...           
124   2023-04-24 00:00:00-04:00
125   2023-04-25 00:00:00-04:00
126   2023-04-26 00:00:00-04:00
127   2023-04-27 00:00:00-04:00
128   2023-04-28 00:00:00-04:00
Name: Date, Length: 129, dtype: datetime64[ns, US/Eastern]

In [24]:
df['Date-Time'].fillna(df_date, inplace = True)


In [25]:
df.tail(6)

,Date-Time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
123,2023-04-21 00:00:00-04:00,285.010010,286.269989,283.059998,285.760010,21665400.0,0.0,0.0,2023-04-21
124,2023-04-24 00:00:00-04:00,282.089996,284.940002,278.720001,281.730011,23781865.0,0.0,0.0,2023-04-24
125,2023-04-25 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-25
126,2023-04-26 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-26
127,2023-04-27 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-27
128,2023-04-28 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-28


In [26]:
df = pd.merge(df, predict_df, how="left", on="Date")


In [27]:
df.tail(6)

,Date-Time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Actual,Predicted
123,2023-04-21 00:00:00-04:00,285.010010,286.269989,283.059998,285.760010,21665400.0,0.0,0.0,2023-04-21,285.76,265.076000
124,2023-04-24 00:00:00-04:00,282.089996,284.940002,278.720001,281.730011,23781865.0,0.0,0.0,2023-04-24,NaN,266.412861
125,2023-04-25 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-25,NaN,266.888085
126,2023-04-26 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-26,NaN,267.182780
127,2023-04-27 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-27,NaN,267.289946
128,2023-04-28 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-28,NaN,267.215979


In [ ]:
# df[~df['Predicted'].isnull().any(axis=1)]
df[df['Predicted']>0]
# df[~df.isnull().any(axis=1)]
